### Generating a toy-case topology

In [1]:
import numpy as np 
import cvxpy as cp

from itertools import combinations
from tqdm import tqdm

In [2]:
# Let's generate a toy topology for our example

nodes = [i for i in range(7)]
edges = [
    (0,1),
    (0,3),
    (0,6),
    (1,2),
    (1,5),
    (2,4),
    (4,6),
    (5,6)
]

V = 7
E = len(edges)

d = 3                                           # Node and edges stalks dimension

F = {
    e:{
        e[0]:np.random.randn(d,d),
        e[1]:np.random.randn(d,d)
        } 
        for e in edges
    }                                           # Incidency linear maps

# Sheaf representation 

# Coboundary map

B = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = F[edge][u]
    B_v = F[edge][v]

    B[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f = B.T @ B

# Generating a smooth signals dataset 

*(from Hansen J., "Learning sheaf Laplacians from smooth signals")* 

In order to retrieve a dataset of smoothsignals, first of all we sample random gaussians vectors on the nodes of the graph. Then we smooth them according to their expansion in terms of the eigenvectors of the sheaf Laplacian $L_0$.

So let's firstly define a dataset of random gaussian vectors. 

In [3]:
N = 100
X = np.random.randn(V*d,N)

Now we'll use the Fourier-domain embedded in the Laplacian spectrum. 

We'll consider a Tikhonov inspired procedure where we firstly project our dataset over the space spanned by the eigenvectors of the sheaf laplacian: namely $U$ the matrix collecting this eigenvectors we have 
\begin{equation}
    \hat{x} = U^T x
\end{equation}

So that defining $h(\lambda) = \frac{1}{1 + 10\lambda}$ and $H = \mathrm{diag}\{h(\lambda)\}_{\lambda}$, we now have

\begin{equation}
    \hat{y} = H(\Lambda) \hat{x}
\end{equation}

and finally our dataset is just reprojected back into the vertex domain:

\begin{equation}
    y = U H(\Lambda) \hat{x} = U H(\Lambda) U^T x
\end{equation}

In [4]:
Lambda, U = np.linalg.eig(L_f)
H = 1/(1 + 10*Lambda)

In [5]:
Y = U @ np.diag(H) @ U.T @ X

Y += np.random.normal(0, 10e-2, size=Y.shape)

In [6]:
np.trace(X.T @ L_f @ X)

16918.49606176653

In [7]:
np.trace(Y.T @ L_f @ Y)

173.17412379035653

____________

In [8]:
def solver(Y, d, edges, N):

    obj = 0
    trace = 0

    Fs = {
        edge: {
            edge[0]:None,
            edge[1]:None
        }
        for edge in edges
    }

    # Loop over edges to define variables and construct objective
    for edge in edges:
        u = edge[0]
        v = edge[1]

        Y_u = Y[u*d:(u+1)*d, :]
        Y_v = Y[v*d:(v+1)*d, :]

        # Define optimization variables for each edge
        Fs[edge][u] = cp.Variable((d, d))
        Fs[edge][v] = cp.Variable((d, d))

        # Update objective and trace
        obj += cp.norm(Fs[edge][u] @ Y_u - Fs[edge][v] @ Y_v, 'fro')**2
        trace += cp.trace(Fs[edge][u]) + cp.trace(Fs[edge][v])

    # Define the trace constraint
    trace_constraint = (trace == N)

    # Define the problem
    problem = cp.Problem(cp.Minimize(obj), [trace_constraint])

    # Solve the problem
    problem.solve(solver=cp.MOSEK,
                  mosek_params = { 'MSK_IPAR_INTPNT_SOLVE_FORM': 'MSK_SOLVE_PRIMAL' },
                  verbose=True)

    # Return the solution
    return Fs

In [28]:
Fs = solver(Y, d, edges, 70)
B_hat = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = Fs[edge][u].value
    B_v = Fs[edge][v].value

    B_hat[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B_hat[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f_hat = B_hat.T @ B_hat
print(f'Average reconstruction error for trace barrier with N = {N}: {np.linalg.norm(L_f - L_f_hat) / L_f.size}')

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Sep 17 12:13:39 PM: Your problem has 144 variables, 1 constraints, and 0 parameters.
(CVXPY) Sep 17 12:13:39 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 17 12:13:39 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 17 12:13:39 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Sep 17 12:13:39 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 17 12:13:39 PM: Compiling problem (target solver=MOSEK).
(C

In [10]:
np.trace(Y.T @ L_f @ Y)

173.17412379035653

In [29]:
np.trace(Y.T @ L_f_hat @ Y) 

127.93240319322629

_______________

In [59]:
def premultiplier(Xu, Xv):
    uu = np.linalg.pinv(Xu @ Xu.T)
    uv = Xu @ Xv.T
    vv = np.linalg.pinv(Xv @ Xv.T)
    vu = Xv @ Xu.T

    return (uu, uv, vv, vu)

def chi_u(uu, uv, vv, vu):

    return chi_v(uu, uv, vv, vu) @ uv - np.eye(uv.shape[0]) @ vv

def chi_v(uu, uv, vv, vu):

    return (vv @ vu @ uu + uu) @ np.linalg.pinv(uv @ vv @ vu @ uu - np.eye(uv.shape[0]))

In [41]:
def premultiplier(Xu, Xv):
    uu = Xu @ Xu.T
    uv = Xu @ Xv.T
    vv = Xv @ Xv.T
    vu = Xv @ Xu.T

    return (uu, uv, vv, vu)

def chi_u(uu, uv, vv, vu):

    return chi_v( uu, uv, vv, vu ) @ (vu + vv) @ np.linalg.pinv( uu + uv)

def chi_v(uu, uv, vv, vu):

    return ( vu + vv ) @ np.linalg.pinv( uu + uv ) @ uu - vu

In [49]:
def premultiplier(Xu, Xv):
    uu = np.linalg.inv(Xu @ Xu.T)
    uv = Xu @ Xv.T
    vv = Xv @ Xv.T
    vu = Xv @ Xu.T

    return (uu, uv, vv, vu)

def chi_u(uu, uv, vv, vu):

    return (uu @ uv + np.eye(uu.shape[0])) @ (np.linalg.inv(vu @ uu @ uv - vv) @ vu - np.eye(uu.shape[0])) @ uu 

def chi_v(uu, uv, vv, vu):

    return (uu @ uv + np.eye(uu.shape[0])) @ (np.linalg.inv(vu @ uu @ uv - vv) @ vu)

In [60]:
T = 0

maps_ = {
    edge : {
        edge[0] : np.zeros((d,d)),
        edge[1] : np.zeros((d,d))
    }
for edge in edges
}

In [61]:
for e in edges:
    
    u = e[0]
    v = e[1]

    X_u = Y[u*d:(u+1)*d,:]
    X_v = Y[v*d:(v+1)*d,:]
    
    uu, uv, vv, vu = premultiplier(X_u, X_v)

    maps_[e][u] = chi_u(uu, uv, vv, vu)
    maps_[e][v] = chi_v(uu, uv, vv, vu)
    
    T += np.trace(maps_[e][u]) + np.trace(maps_[e][v])

In [62]:
maps_

{(0,
  1): {0: array([[-2.51920109, -1.49194897, -1.48330757],
         [ 1.6049498 ,  0.54502809,  1.53393783],
         [-1.13473406, -1.05805019, -1.74397252]]), 1: array([[-1.15664792,  0.46424933, -0.05850676],
         [ 0.01242712, -0.86870764,  0.10292434],
         [-0.13340224,  0.63958609, -0.88124059]])},
 (0,
  3): {0: array([[-0.46567798,  0.00303673,  0.45687514],
         [-0.38395581, -0.57042597, -0.26470322],
         [ 0.19754794, -0.1919172 , -0.38220928]]), 3: array([[-0.65111127, -0.28827351,  0.13499319],
         [-0.03782091, -0.58006407,  0.08199867],
         [ 0.28429744,  0.10689501, -0.81985981]])},
 (0,
  6): {0: array([[-1.4611331 ,  0.27734304, -1.16792285],
         [ 0.90708254, -0.99346138,  1.56811421],
         [ 0.03431326,  0.02139971, -0.09490457]]), 6: array([[-0.98455901,  0.24196038,  0.1867123 ],
         [ 0.14789289, -0.86269656, -0.02660246],
         [ 0.13485008,  0.17735056, -0.8644512 ]])},
 (1,
  2): {1: array([[-0.92553332, -0.1018

In [63]:
maps_ = {
    edge : {
        edge[0] : 70/T * maps_[edge][edge[0]],
        edge[1] : 70/T * maps_[edge][edge[1]]
    }
for edge in edges
}

In [64]:
B_hat_2 = np.zeros((d*E, d*V))

for i, edge in enumerate(edges):

    u = edge[0] 
    v = edge[1] 

    B_u = maps_[edge][u]
    B_v = maps_[edge][v]

    B_hat_2[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B_hat_2[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f_hat_2 = B_hat_2.T @ B_hat_2

In [65]:
np.trace(Y.T @ L_f_hat_2 @ Y)

707.4818824779823

In [66]:
maps_

{(0,
  1): {0: array([[ 3.71577626,  2.20059788,  2.18785197],
         [-2.36727206, -0.80390662, -2.2625307 ],
         [ 1.67371231,  1.56060498,  2.57232808]]), 1: array([[ 1.70603487, -0.68475941,  0.08629642],
         [-0.01832978,  1.28132813, -0.15181155],
         [ 0.19676591, -0.94337797,  1.29981401]])},
 (0,
  3): {0: array([[ 0.68686663, -0.00447913, -0.67388261],
         [ 0.56632791,  0.84136804,  0.39043248],
         [-0.29137966,  0.28307442,  0.56375181]]), 3: array([[ 0.96037741,  0.42519824, -0.19911253],
         [ 0.05578516,  0.85558406, -0.12094657],
         [-0.41933361, -0.15766822,  1.20927846]])},
 (0,
  6): {0: array([[ 2.15514503, -0.409076  ,  1.72266518],
         [-1.33793042,  1.46533766, -2.31294023],
         [-0.05061144, -0.03156419,  0.13998253]]), 6: array([[ 1.45220682, -0.35688721, -0.27539728],
         [-0.21813935,  1.2724619 ,  0.03923815],
         [-0.19890144, -0.26158889,  1.27504996]])},
 (1,
  2): {1: array([[ 1.36514499,  0.1502

In [67]:
Fs[(0,1)][0].value

array([[ 2.11392386, -0.84847578,  0.10692868],
       [-0.02271218,  1.5876757 , -0.18810756],
       [ 0.24380987, -1.1689264 ,  1.61058129]])